## Initialization

In [1]:
import matplotlib.pyplot as plt
from mog_experiment import *
%autoreload 2

In [2]:
X_DIM = 3
NUM_DISTRIBUTIONS = 3

MEANS = np.zeros((X_DIM, X_DIM), dtype=np.float32)
CENTERS = np.sqrt(2)  # center of Gaussian
for i in range(NUM_DISTRIBUTIONS):
    MEANS[i::NUM_DISTRIBUTIONS, i] = CENTERS

params = {                          # default parameter values
    'x_dim': X_DIM,
    'num_distributions': NUM_DISTRIBUTIONS,
    'means': MEANS,
    'sigma': 0.05,
    'small_pi': 2E-16,
    'scale': 0.1,
    'num_samples': 200,
    'lr_init': 1e-3,
    'lr_decay_steps': 1000,
    'eps': 0.3,
    'temp_init': 15,
    'annealing_steps': 150,
    'annealing_rate': 0.98,
    'train_trajectory_length': 15,
    'test_trajectory_length': 2000,
    'num_training_steps': 30000,
    'tunneling_rate_steps': 200,
    'save_steps': 2500,
    'logging_steps': 100
}

## Build / Train model

In [11]:
tf.reset_default_graph()

In [12]:
#%pdb
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
model = GaussianMixtureModel(params, 
                             config=config,
                             log_dir='../log_mog_tf/run87/')
                             #log_dir='../log_mog_tf/run64/')

Loading from previous parameters in from: ../log_mog_tf/run87/run_info/
params file written to: ../log_mog_tf/run87/run_info/parameters.txt


In [14]:
model.sess.run(tf.global_variables_initializer())

In [16]:
model.tunneling_rates_highT.keys()

dict_keys([(999, 13.022), (1999, 11.305), (2999, 10.014), (3999, 8.694), (4999, 7.547), (5999, 6.686), (6999, 5.804), (7999, 5.353), (8999, 4.742), (9999, 4.117), (10999, 3.574), (11999, 3.166), (12999, 2.748), (13999, 2.386)])

In [19]:
for key in model.tunneling_rates_highT.keys():
    #print(f'Step num: {key[0]}')
    #print(f'Temp: {key[1]}')
    model.steps_arr.append(key[0])
    model.temp_arr.append(key[1])
    #model.temp_arr.append(key[1])

In [24]:
for idx in range(len(model.steps_arr)):
    model.steps_arr[idx] += 1

In [25]:
model.steps_arr

[1000,
 2000,
 3000,
 4000,
 5000,
 6000,
 7000,
 8000,
 9000,
 10000,
 11000,
 12000,
 13000,
 14000]

In [22]:
for key, val in model.tunneling_rates.items():
    model.tunneling_rates_avg.append(val[0])
    model.tunneling_rates_err.append(val[1])
for key, val in model.tunneling_rates_highT.items():
    model.tunneling_rates_avg_highT.append(val[0])
    model.tunneling_rates_err_highT.append(val[1])
    
for key, val in model.acceptance_rates.items():
    model.acceptance_rates_avg.append(val[0])
    model.acceptance_rates_err.append(val[1])
for key, val in model.acceptance_rates_highT.items():
    model.acceptance_rates_avg_highT.append(val[0])
    model.acceptance_rates_err_highT.append(val[1])
    
for key, val in model.distances.items():
    model.distances_avg.append(val[0])
    model.distances_err.append(val[1])
for key, val in model.distances_highT.items():
    model.distances_avg_highT.append(val[0])
    model.distances_err_highT.append(val[1])

In [27]:
model._save_variables()

Saving parameter values to: ../log_mog_tf/run87/run_info/


In [32]:
model._init_params(params)

In [37]:
model._load_variables()

Loading from previous parameters in from: ../log_mog_tf/run87/run_info/


In [38]:
model.steps_arr

[]

In [41]:
model.tunneling_rates

{(999, 1.0): (0.0012481240620310155, 5.8331027772798283e-05),
 (1999, 1.0): (0.0034267133566783393, 8.140345708493158e-05),
 (2999, 1.0): (0.005477738869434717, 0.00013949275563455405),
 (3999, 1.0): (0.008394197098549274, 0.00017168694712826115),
 (4999, 1.0): (0.011680840420210106, 0.0002233188021439273),
 (5999, 1.0): (0.020572786393196596, 0.0003098669870569123),
 (6999, 1.0): (0.02786393196598299, 0.00037911459430773286),
 (7999, 1.0): (0.0313031515757879, 0.00034142907767175935),
 (8999, 1.0): (0.03554777388694347, 0.00044896933285469245),
 (9999, 1.0): (0.059582291145572785, 0.0007551928360087015),
 (10999, 1.0): (0.07291395697848925, 0.0006813746588816416),
 (11999, 1.0): (0.11760130065032516, 0.0008851670448373469),
 (12999, 1.0): (0.1176438219109555, 0.0007716030344731442),
 (13999, 1.0): (0.18985492746373187, 0.0008756115494580632)}

In [42]:
import os

In [47]:
model.__dict__.keys()

dict_keys(['params', 'means', 'covs', 'distribution', 'samples', 'tunneling_rates', 'tunneling_rates_highT', 'tunneling_rates_avg', 'tunneling_rates_avg_highT', 'tunneling_rates_err', 'tunneling_rates_err_highT', 'acceptance_rates', 'acceptance_rates_highT', 'acceptance_rates_avg', 'acceptance_rates_avg_highT', 'acceptance_rates_err', 'acceptance_rates_err_highT', 'distances', 'distances_highT', 'distances_avg', 'distances_avg_highT', 'distances_err', 'distances_err_highT', 'losses_arr', 'temp_arr', 'steps_arr', 'temp', 'attrs_dict', 'log_dir', 'info_dir', 'figs_dir', 'params_file', 'distances_file', 'tunneling_rates_file', 'acceptance_rates_file', 'distances_highT_file', 'tunneling_rates_highT_file', 'acceptance_rates_highT_file', 'global_step', 'learning_rate', 'dynamics', 'x', 'z', 'Lx', 'px', 'output', 'Lz', 'pz', 'loss', 'train_op', 'summary_op', 'sess'])

In [53]:
get_vals_as_arr = lambda _dict: np.array(list(_dict.values()))
tr = get_vals_as_arr(model.tunneling_rates)
ar = get_vals_as_arr(model.acceptance_rates)
dr = get_vals_as_arr(model.distances)
#np.array(list(model.tunneling_rates.values()))

In [45]:
for key in model.attrs_dict.keys():
    in_file = model.info_dir + key + '.npy'
    if os.path.isfile(in_file):
        setattr(model, key, np.load(in_file))
        print(f'Set model.{key} to values read in from: {in_file}')
        #print(model.key == np.load(in_file))

Set model.losses_arr to values read in from: ../log_mog_tf/run87/run_info/losses_arr.npy
Set model.steps_arr to values read in from: ../log_mog_tf/run87/run_info/steps_arr.npy
Set model.temp_arr to values read in from: ../log_mog_tf/run87/run_info/temp_arr.npy
Set model.means to values read in from: ../log_mog_tf/run87/run_info/means.npy
Set model.covs to values read in from: ../log_mog_tf/run87/run_info/covs.npy
Set model.tunneling_rates_avg to values read in from: ../log_mog_tf/run87/run_info/tunneling_rates_avg.npy
Set model.tunneling_rates_err to values read in from: ../log_mog_tf/run87/run_info/tunneling_rates_err.npy
Set model.tunneling_rates_avg_highT to values read in from: ../log_mog_tf/run87/run_info/tunneling_rates_avg_highT.npy
Set model.tunneling_rates_err_highT to values read in from: ../log_mog_tf/run87/run_info/tunneling_rates_err_highT.npy
Set model.acceptance_rates_avg to values read in from: ../log_mog_tf/run87/run_info/acceptance_rates_avg.npy
Set model.acceptance_r

In [35]:
model.sess.run(tf.global_variables_initializer())

In [28]:
attrs_dict = model.attrs_dict

In [31]:
attrs_dict['steps_arr']

[1000,
 2000,
 3000,
 4000,
 5000,
 6000,
 7000,
 8000,
 9000,
 10000,
 11000,
 12000,
 13000,
 14000]

In [19]:
model.train(params['num_training_steps'], plot=True)

Step: 0/30000, Loss: -149.3, accept rate: 0.97, LR: 0.001, temp: 25, step size: 0.3

Step: 100/30000, Loss: -218.9, accept rate: 0.96, LR: 0.001, temp: 24.5, step size: 0.278


	TEMP: 1, Step: 200, Loss (avg): 23.51, 
	Accept rate (avg): 0.7757, Accept rate (err): 1.959e-07, 
	Tunneling rate (avg): 0.0003627, Tunneling rate (err): 1.508e-07


	TEMP: 24, Step: 200, Loss (avg): -231.8, 
	Accept rate (avg): 0.9475, Accept rate (err): 6.368e-08, 
	Tunneling rate (avg): 0.5256, Tunneling rate (err): 6.875e-06


	Time to calculate tunneling_rate: 00:02:17
	Time for 100 training steps: 00:01:48
	Total time elapsed: 00:03:35

Saving parameter values to: ../log_mog_tf/run78/run_info/
done!

Saving figure to: ../log_mog_tf/run78/figures/tr_ar_steps_lowT_200.pdf
Saving figure to: ../log_mog_tf/run78/figures/tr_ar_steps_highT_200.pdf
Saving figure to: ../log_mog_tf/run78/figures/tr_ar_temps_lowT_200.pdf
Saving figure to: ../log_mog_tf/run78/figures/tr_ar_temps_highT_200.pdf
Step: 200/30000, Loss: 

## OLD

In [ ]:
#trajectories, loss_arr, px_arr = model.generate_trajectories(num_samples=100, num_steps=100, temperature=1.)

#config = tf.ConfigProto(log_device_placement=True)
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#model.build_graph()

In [ ]:
def calc_avg_distance1(trajectories):
    distances_arr = []
    for trajectory in trajectories:
        distance_arr = []
        for idx in range(1, len(trajectory)):
            diff = trajectory[idx] - trajectory[idx-1]
            dist = np.sqrt(np.dot(diff, diff.T))
            distance_arr.append(dist)
        distances_arr.append(sum(distance_arr))
    return np.mean(distances_arr)

def calc_avg_distance2(trajectories):
    dist = lambda d: np.sqrt(np.dot(d, d.T))
    #distances_arr = np.mean([[dist(d) for d in [t[:-1, :] - t[1:, :] for t in trajectories]]
    #for trajectory in trajectories:
        diff = trajectory[:-1, :] - trajectory[1:, :]
        distance = sum([np.sqrt(np.dot(d, d.T)) for d in diff])
        distances_arr.append(distance)
    return np.mean(distances_arr)

In [ ]:
%timeit avg_dist = avg_distance_traveled(trajectories.transpose([1,0,2]))
#print(avg_dist)

%timeit avg_dist1 = calc_avg_distance1(trajectories.transpose([1,0,2]))
#print(avg_dist1)

%timeit avg_dist2 = calc_avg_distance2(trajectories.transpose([1,0,2]))
#print(avg_dist2)

distances = np.array([np.sqrt(np.dot(disp, disp.T)) for disp in displacements])

In [ ]:
model.train(params['num_training_steps'], config=config, plot=True)

config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
model.build_graph()
model.train(params['num_training_steps'], config=config, plot=False)